In [1]:
import wandb
from wandb.keras import WandbCallback
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Masking, Bidirectional
from keras.callbacks import EarlyStopping
from keras import utils
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt

In [2]:
trainX = pd.read_csv('./Train/trainX3.csv')
trainY = pd.read_csv('./Train/trainY3.csv')
testX = pd.read_csv('./Test/testX3.csv')
testY = pd.read_csv('./Test/testY3.csv')

In [ ]:
# Create model

def plot_roc_curve(fpr,tpr): 
  plt.plot(fpr,tpr) 
  plt.axis([0,1,0,1]) 
  plt.xlabel('False Positive Rate') 
  plt.ylabel('True Positive Rate') 
  plt.show()    
  

for dropout in [0.1]:
    wandb.init(reinit=True)
    print("\n\nTesting model with dropout = %f\n"%dropout)
    model = Sequential()
    model.add(Masking(mask_value=-10, input_shape=(541,1)))
    model.add(Bidirectional(LSTM(units=32)))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))
    
    checkpoint_filepath = 'weights.{epoch:02d}-{accuracy:.2f}-{val_accuracy:.2f}.h5'
    
    model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        monitor='accuracy',
        mode='max',
        save_best_only=True)
    
    #early_stopper = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, restore_best_weights=False)
        
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy','AUC'])
    history = model.fit(trainX, trainY, epochs=2000, batch_size=81, validation_split=0.1, verbose=1, callbacks= [model_checkpoint_callback])
    # Plot training & validation accuracy values
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val', 'Loss', 'Val_Loss'], loc='upper left')
    plt.show()
    wandb.save('model.h5')
    wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ashoka (use `wandb login --relogin` to force relogin)




Testing model with dropout = 0.100000

Epoch 1/2000
1/1 [==============================] - 10s 10s/step - loss: 0.7063 - accuracy: 0.4430 - auc: 0.6206 - val_loss: 0.6715 - val_accuracy: 0.6667 - val_auc: 0.6667
Epoch 2/2000
1/1 [==============================] - 0s 102ms/step - loss: 0.6966 - accuracy: 0.4304 - auc: 0.6578 - val_loss: 0.6894 - val_accuracy: 0.6667 - val_auc: 0.0833
Epoch 3/2000
1/1 [==============================] - 0s 102ms/step - loss: 0.6987 - accuracy: 0.4177 - auc: 0.4529 - val_loss: 0.7083 - val_accuracy: 0.3333 - val_auc: 0.0833
Epoch 4/2000
1/1 [==============================] - 0s 124ms/step - loss: 0.6921 - accuracy: 0.5190 - auc: 0.4516 - val_loss: 0.7274 - val_accuracy: 0.3333 - val_auc: 0.0833
Epoch 5/2000
1/1 [==============================] - 0s 103ms/step - loss: 0.6832 - accuracy: 0.5949 - auc: 0.5716 - val_loss: 0.7460 - val_accuracy: 0.3333 - val_auc: 0.0833
Epoch 6/2000
1/1 [==============================] - 0s 84ms/step - loss: 0.6890 - accuracy

In [ ]:
model.load_weights('./weights.128-0.86-0.78.h5')

import sklearn
from sklearn.metrics import roc_curve,roc_auc_score

prediction_prob = np.round(model.predict(testX),2)
validation = testY.values.tolist()
validation = np.array(validation)
validation = validation.flatten()
auc_score=roc_auc_score(validation,prediction_prob)
print("With a dropout of",dropout,"The AUC score is {:0.2f}".format(auc_score),"on test set")
#wandb.log({"ROC":plot_roc_curve(fpr,tpr)})   
fpr , tpr , thresholds = roc_curve (validation , prediction_prob)
plot_roc_curve(fpr,tpr)

In [ ]:
comparison = {'prob': prediction_prob.flatten(), 'label': validation}
comparison = pd.DataFrame(comparison)
comp = comparison.sort_values(by='prob')
pd.set_option('display.max_columns', None)
comp.T

In [ ]:
from sklearn.metrics import brier_score_loss

loss = brier_score_loss(comp['label'],comp['prob'])
prediction = list(map(lambda x: 0 if x<0.5 else 1, prediction_prob))

print("With a dropout of 0.1, The AUC score is {:0.2f}".format(auc_score),"on test set")
print("The accuracy on test set is {:0.2f}".format(sum([1 for i in range(len(prediction)) if prediction[i] == validation[i]])/len(prediction)*100))
print('The Brier Score on test set is {:0.2f}.' .format(loss))